In [ ]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

# Automated Warehouse Management

## Description of the use case
This use case involves automating warehouse management by tracking inventory levels, managing restocking processes, and optimizing storage locations. By using real-time data, businesses can ensure efficient warehouse operations.

## Redis Data Structures Used
- **Hashes**: To store inventory and product information.
- **Streams**: To capture and process real-time inventory updates.
- **Sorted Sets**: To prioritize restocking tasks based on inventory levels.


In [ ]:
# Mock data for the use case
inventory_data = {
    'product_1001': {'name': 'Product A', 'quantity': 50},
    'product_1002': {'name': 'Product B', 'quantity': 20},
    'product_1003': {'name': 'Product C', 'quantity': 100},
    'product_1004': {'name': 'Product D', 'quantity': 10}
}

# Use Hashes to store inventory and product information
for product_id, data in inventory_data.items():
    r.hset(product_id, mapping=data)
    
print("Before updates:")
for product_id in inventory_data.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")

# Use Streams to capture real-time inventory updates
r.xadd('inventory_updates', {'product_id': 'product_1001', 'quantity': -5})
r.xadd('inventory_updates', {'product_id': 'product_1002', 'quantity': -10})

# Function to process inventory updates and adjust stock levels
def process_inventory_updates():
    stream_entries = r.xrange('inventory_updates')
    for entry in stream_entries:
        product_id = entry[1]['product_id']
        quantity_change = int(entry[1]['quantity'])
        current_quantity = int(r.hget(product_id, 'quantity'))
        new_quantity = current_quantity + quantity_change
        r.hset(product_id, 'quantity', new_quantity)
        if new_quantity < 20:  # Example threshold for restocking
            r.zadd('restocking_priority', {product_id: new_quantity})

# Process the inventory updates
process_inventory_updates()

# Print the updated inventory data
print("After updates:")
for product_id in inventory_data.keys():
    print(f"{product_id}: {r.hgetall(product_id)}")

# Print the restocking priorities
restocking_priorities = r.zrange('restocking_priority', 0, -1, withscores=True)
print('Restocking Priorities:')
for product in restocking_priorities:
    print(product)
